In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import socket

In [26]:
path_to_dir = '/d/DATD/datd/MD_TMS_EEG/calevents'
evtfname = 'events-export.csv'
evtfpath = os.path.join(path_to_dir, evtfname)

df = pd.read_csv(evtfpath)
df.loc[df['Invitee Name'] == 'QIN HE', 'Invitee Name'] = 'Richard He'
df.loc[df['Invitee Name'] == 'Qingqing', 'Invitee Name'] = 'Qingqing Yang'
df.loc[df['Invitee Name'] == 'Eve', 'Invitee Name'] = 'Eve Yan'
df.loc[df['Invitee Name'] == 'Zoe', 'Invitee Name'] = 'Zoe Duan'
df.loc[df['Invitee Name'] == 'Nalani Lalita Saxena Mumm', 'Invitee Name'] = 'Nalani Mumm'
df.loc[df['Invitee Name'] == 'Madhura T Ravindra ', 'Invitee Name'] = 'Madhura Ravindra'
df.loc[df['Invitee Name'] == 'Madhura Ravindra ', 'Invitee Name'] = 'Madhura Ravindra'
df.loc[df['Invitee Name'] == 'Fangfang hong', 'Invitee Name'] = 'Fangfang Hong'
df.loc[df['Invitee Name'] == 'Luhe ', 'Invitee Name'] = 'Luhe'
df.loc[df['Invitee Name'] == 'Mihir Hargudkar ', 'Invitee Name'] = 'Mihir Hargudkar'


df['Start Date & Time'] = pd.to_datetime(df['Start Date & Time'])
count_dates = df.groupby(['Event Type Name', 'Invitee Name'])['Start Date & Time'].count().reset_index()
last_dates = df.groupby(['Event Type Name', 'Invitee Name'])['Start Date & Time'].agg(list).reset_index()

/var/folders/cf/d9j3_ppd5wb4bn9y_fgj3nb800010l/T/ipykernel_47808/3255083270.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start Date & Time'] = pd.to_datetime(df['Start Date & Time'])


In [22]:
last_dates['Event Type Name'].unique()

array(['TMS Phosphene Mapping', 'TMS vWM Control', 'TMS-EEG vWM'],
      dtype=object)

In [51]:
date_tracker = pd.DataFrame(columns=['Invitee Name', 'Control Date', 'Main Exp Dates', 'Difference'])
for sub in last_dates['Invitee Name'].unique():
    df_sub = last_dates[last_dates['Invitee Name'] == sub]
    if 'TMS vWM Control' in df_sub['Event Type Name'].values:
        control_date = df_sub[df_sub['Event Type Name'] == 'TMS vWM Control']['Start Date & Time'].values
        main_exp_dates = df_sub[df_sub['Event Type Name'] == 'TMS-EEG vWM']['Start Date & Time'].values
        
        diff = control_date[0][0] - main_exp_dates[0][-1]
        # print(control_date, main_exp_dates, diff)
        date_tracker = pd.concat([date_tracker, pd.DataFrame({'Invitee Name': sub, 'Control Date': control_date, 'Main Exp Dates': main_exp_dates, 'Difference': diff})], ignore_index=True)
        # date_tracker = date_tracker.append({'Invitee Name': sub, 'Control Date': control_date, 'Main Exp Dates': main_exp_dates, 'Difference': diff}, ignore_index=True)
    else:
        print(f'{sub} does not have a control date')
        main_exp_dates = df_sub[df_sub['Event Type Name'] == 'TMS-EEG vWM']['Start Date & Time'].values
        phosphMap_dates = df_sub[df_sub['Event Type Name'] == 'TMS Phosphene Mapping']['Start Date & Time'].values
        if len(main_exp_dates) > 0:
            if len(main_exp_dates[0]) >= 3 and len(phosphMap_dates) != 0:
                diff = main_exp_dates[0][-1] - main_exp_dates[0][-2]
                date_tracker = pd.concat([date_tracker, pd.DataFrame({'Invitee Name': sub, 'Control Date': np.nan, 'Main Exp Dates': main_exp_dates, 'Difference': diff})], ignore_index=True)    

Fangfang Hong does not have a control date
Luhe does not have a control date
Madhura Ravindra does not have a control date
Manasi Khandekar does not have a control date
Mariam Beriashvili does not have a control date
Rania does not have a control date
Shutian Xue does not have a control date
Tomas Alegria does not have a control date
Yichen Lu does not have a control date
Yuyang Xu does not have a control date
zhenming Liu does not have a control date
clay does not have a control date


/var/folders/cf/d9j3_ppd5wb4bn9y_fgj3nb800010l/T/ipykernel_47808/635626048.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  date_tracker = pd.concat([date_tracker, pd.DataFrame({'Invitee Name': sub, 'Control Date': control_date, 'Main Exp Dates': main_exp_dates, 'Difference': diff})], ignore_index=True)


In [54]:
date_tracker[date_tracker['Invitee Name'] != 'Rania']

,Invitee Name,Control Date,Main Exp Dates,Difference
0,David Tu,[2023-12-08 14:15:00],"[2023-08-15 13:00:00, 2023-08-16 13:00:00, 202...",113 days 01:15:00
1,Eve Yan,[2023-12-12 12:30:00],"[2023-08-18 13:00:00, 2023-08-21 11:00:00, 202...",110 days 02:30:00
2,Hsing-Hao Lee,[2023-12-12 10:30:00],"[2023-08-15 10:00:00, 2023-08-18 10:00:00, 202...",111 days 00:30:00
3,Jacob Serfaty,[2023-12-05 14:00:00],"[2023-05-14 14:00:00, 2023-05-16 14:30:00, 202...",199 days 00:30:00
4,Luhe,NaN,"[2023-10-19 16:00:00, 2023-10-20 16:30:00, 202...",1 days 00:00:00
5,Madhura Ravindra,NaN,"[2023-08-07 14:00:00, 2023-08-08 14:00:00, 202...",1 days 00:00:00
6,Marc Himmelberg,[2023-12-08 18:15:00],"[2023-05-15 17:00:00, 2023-05-26 16:00:00, 202...",192 days 02:15:00
7,Mihir Hargudkar,"[2023-12-03 17:00:00, 2023-12-06 11:15:00]","[2023-08-14 16:00:00, 2023-08-15 16:00:00, 202...",109 days 01:00:00
8,Qingqing Yang,[2023-12-07 15:45:00],"[2023-04-19 16:00:00, 2023-04-24 16:00:00, 202...",196 days 00:45:00
9,Rachel Chen,[2023-12-03 13:00:00],"[2023-05-03 17:00:00, 2023-05-04 16:30:00, 202...",211 days 21:00:00


In [55]:
date_tracker[date_tracker['Invitee Name'] != 'Rania']['Difference'].mean()

Timedelta('110 days 01:20:17.647058824')

In [19]:

# Get the last date for each subject by event type
for exp in df['Event Type Name'].unique():
    df_exp = df[df['Event Type Name'] == exp]
    sub_list = df_exp['Invitee Name'].unique()
    for sub in sub_list:
        df_sub = df_exp[df_exp['Invitee Name'] == sub]
        this_sub_dates = df_sub['Start Date & Time']
        dt_list = []
        for date in this_sub_dates:
            dt_this = pd.to_datetime(date[:10])
            dt_list.append(dt_this)
        print(dt_list)
        # print(exp, sub, this_sub_dates, len(this_sub_dates))

[Timestamp('2023-04-04 00:00:00')]
[Timestamp('2023-04-06 00:00:00')]
[Timestamp('2023-04-14 00:00:00')]
[Timestamp('2023-04-20 00:00:00')]
[Timestamp('2023-04-23 00:00:00')]
[Timestamp('2023-05-08 00:00:00')]
[Timestamp('2023-05-01 00:00:00')]
[Timestamp('2023-05-05 00:00:00')]
[Timestamp('2023-05-09 00:00:00')]
[Timestamp('2023-05-08 00:00:00')]
[Timestamp('2023-05-11 00:00:00')]
[Timestamp('2023-05-11 00:00:00')]
[Timestamp('2023-07-22 00:00:00'), Timestamp('2023-07-31 00:00:00')]
[Timestamp('2023-07-21 00:00:00')]
[Timestamp('2023-07-21 00:00:00')]
[Timestamp('2023-08-11 00:00:00'), Timestamp('2023-08-14 00:00:00'), Timestamp('2023-08-29 00:00:00'), Timestamp('2023-08-30 00:00:00'), Timestamp('2023-08-31 00:00:00')]
[Timestamp('2023-08-03 00:00:00'), Timestamp('2023-08-09 00:00:00')]
[Timestamp('2023-07-24 00:00:00')]
[Timestamp('2023-07-27 00:00:00')]
[Timestamp('2023-07-30 00:00:00'), Timestamp('2023-08-10 00:00:00')]
[Timestamp('2023-07-28 00:00:00')]
[Timestamp('2023-08-07 00:0